In [1]:
base_dir = '/home/shaeo/cadd_training/20250915_nvmolkit'
nvmolkit_dir = '/home/shaeo/opt/nvMolKit'

In [2]:
import os
import time
from rdkit import Chem
from rdkit.Chem.rdDistGeom import ETKDGv3
from nvmolkit.embedMolecules import EmbedMolecules as nvMolKitEmbed
from nvmolkit.types import HardwareOptions
from nvmolkit.mmffOptimization import MMFFOptimizeMoleculesConfs as nvMolKitMMFFOptimize

In [6]:
# Configuration
SDF_FILE = nvmolkit_dir + '/benchmarks/data/MPCONF196.sdf'
MAX_MOLECULES = 50
CONFORMERS_PER_MOLECULE = 5
RANDOM_SEED = 42

In [7]:
# Setup ETKDG parameters
params = ETKDGv3()
params.randomSeed = RANDOM_SEED
params.useRandomCoords = True  # Required for nvMolKit

## SDFから分子を読み込む

In [8]:
if not os.path.exists(SDF_FILE):
    raise FileNotFoundError(f"SDF file not found: {SDF_FILE}")

supplier = Chem.SDMolSupplier(SDF_FILE, removeHs=False, sanitize=True)
molecules = []

for i, mol in enumerate(supplier):
    if mol is None:
        continue
    if i >= MAX_MOLECULES:
        break
    
    # Clear any existing conformers for clean embedding tests
    mol.RemoveAllConformers()
    molecules.append(mol)

print(f"Successfully loaded {len(molecules)} molecules from {SDF_FILE}")

Successfully loaded 50 molecules from /home/stake/opt/nvMolKit/benchmarks/data/MPCONF196.sdf


## コンフォメーション生成：GPU

In [9]:
hardware_opts = HardwareOptions(
    preprocessingThreads=2,
    batchSize=25,
    batchesPerGpu=2,
)

start_time = time.time()

nvMolKitEmbed(
    molecules=molecules,
    params=params,
    confsPerMolecule=CONFORMERS_PER_MOLECULE,
    maxIterations=-1,  # Automatic iteration calculation
    hardwareOptions=hardware_opts
)

embedding_time = time.time() - start_time
total_conformers = sum(mol.GetNumConformers() for mol in molecules)

print(f"Conformer generation completed in {embedding_time:.2f} seconds")
print(f"Generated {total_conformers} total conformers")
print(f"Rate: {total_conformers/embedding_time:.1f} conformers/second")


Conformer generation completed in 5.78 seconds
Generated 233 total conformers
Rate: 40.3 conformers/second


## コンフォメーション最適化：GPU

In [10]:
mmff_hardware_opts = HardwareOptions(
    preprocessingThreads=4,
    batchSize=0,  # Process all conformers together
)

total_conformers = sum(mol.GetNumConformers() for mol in molecules)
start_time = time.time()

energies = nvMolKitMMFFOptimize(
    molecules=molecules,
    maxIters=200,
    nonBondedThreshold=100.0,
    hardwareOptions=mmff_hardware_opts
)

optimization_time = time.time() - start_time

print(f"MMFF optimization completed in {optimization_time:.2f} seconds")
print(f"Rate: {total_conformers/optimization_time:.1f} conformers/second")

MMFF optimization completed in 1.32 seconds
Rate: 177.1 conformers/second


## コンフォメーション生成：CPU

In [20]:
from rdkit.Chem import AllChem

In [22]:
params = AllChem.ETKDGv3()
params.numThreads = 16
params.randomSeed = RANDOM_SEED
params.useRandomCoords = True

In [23]:
# コンフォメーションをリセット
for mol in molecules:
    mol.RemoveAllConformers()

start_time = time.time()

for mol in molecules:
    AllChem.EmbedMultipleConfs(mol, numConfs=5, params=AllChem.ETKDGv3())

embedding_time = time.time() - start_time
total_conformers = sum(mol.GetNumConformers() for mol in molecules)

print(f"Conformer generation completed in {embedding_time:.2f} seconds")
print(f"Generated {total_conformers} total conformers")
print(f"Rate: {total_conformers/embedding_time:.1f} conformers/second")


Conformer generation completed in 96.09 seconds
Generated 248 total conformers
Rate: 2.6 conformers/second


## コンフォメーション最適化：CPU

In [25]:
total_conformers = sum(mol.GetNumConformers() for mol in molecules)
start_time = time.time()

for mol in molecules:
    results = AllChem.MMFFOptimizeMoleculeConfs(
        mol,
        numThreads=16,
        maxIters=200,
        nonBondedThresh=100.0,
    )

optimization_time = time.time() - start_time

print(f"MMFF optimization completed in {optimization_time:.2f} seconds")
print(f"Rate: {total_conformers/optimization_time:.1f} conformers/second")

MMFF optimization completed in 4.43 seconds
Rate: 56.0 conformers/second
